In [1]:
import pyvista as pv

import random as rd
import numpy as np 
from scipy.sparse import *

from matplotlib.colors import ListedColormap

import clearDatatool
import manifold
import topologyProcess
import conformalProcess

#Topological and conformal translating for 2D closed manifolds

In [2]:
# read a mesh from .ply or .obj
mesh = pv.read('datasource/kitten.ply')
# clean the mesh data to guarantee the regulation of data to construct manifold 
Points, Faces = clearDatatool.clearMesh(mesh)

In [3]:
# construct 2D manifold
m2 = manifold.Manifold2D(Points,Faces)

# convert a manifold into a chain complex 
c2 = m2.toChain()

# search the boundary of the manifold 
boundary = m2.boundary()
print('m2 has ' + str(len(boundary))+' boundary(ies)')



orientated
closed manifold without boundary
m2 has 0 boundary(ies)


In [4]:
# calculate bases of the fundamental Group for a closed manifold 
baseloops = topologyProcess.fundamentalGroup(m2)
print('The fundamental group Pi_1(m2) has ' + str(len(baseloops))+ ' bases')

The fundamental group Pi_1(m2) has 2 bases


In [5]:
# visualize the bases of fundamental Group (or 1-dimensional homology group H_1)
Pic = pv.Plotter()
Pic.add_mesh(m2.toMesh())
colortem = ['blue','red']
for i in range(len(baseloops)):
    loop = baseloops[i]
    showline = np.array([loop[0].startP.pos]+[HE.endP.pos for HE in loop])
    Pic.add_lines(showline, color=colortem[i], width=4)
Pic.show()




ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [6]:
# By cutting the manifold along fundamental bases, we obtain principal domain D_2
cuttrace = set()
for loop in baseloops:
    cuttrace = cuttrace.union(loop)


In [7]:
# construct cohomolpgy bases for the manifold
cohomBases = [topologyProcess.toCochain(c2,loop) for loop in baseloops]
### cohomBases = topologyProcess.cohomolpogyBases(m2)
print(cohomBases)

[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]


In [8]:
# calculate the harmonic bases for the manifold
Hbases = [conformalProcess.HodgeDecompose(m2,coloop,onlyH=True) for coloop in cohomBases]
###Hbases = conformalProcess.HodgeBase_dim1(m2)
print(Hbases)

[array([ 5.62777706e-04, -1.29237123e-03, -7.62308200e-04, ...,
       -2.49125994e-03,  3.35511639e-04, -7.38707529e-05]), array([-1.63268280e-05, -1.90453675e-04,  5.52688065e-05, ...,
       -1.93373011e-04,  3.83197221e-05, -1.66875178e-04])]


In [9]:
# integral of harmonic bases on principle domain
intH = [topologyProcess.intergalCochain(H,m2,cuttrace) for H in Hbases] 
# visualize the harmonic bases
Pic = pv.Plotter(shape=(int(len(intH)/2),2))

black = np.array([11 / 256, 11 / 256, 11 / 256, 1])
white = np.array([0.92, 0.92, 0.92, 1])
my_colormap = ListedColormap([black,white]*61)

for i in range(len(intH)):
    Pic.subplot(int(i/2),i%2)
    meshtem = mesh.copy()
    meshtem.point_data['integral'+str(i)] = intH[i]
    Pic.add_mesh(meshtem, scalars='integral'+str(i),style='surface',cmap=my_colormap)

Pic.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Conformal transport on topological Desk  

In [10]:
# dig a hole on a human face (topological desk)
mesh = pv.read('datasource/girl_open.ply')
# clean data roughly
Points, Faces = clearDatatool.clearMesh_easy(mesh)
Ntem = 4000
Faces = Faces[:Ntem]+Faces[Ntem+1:]
m2 = manifold.Manifold2D(Points,Faces)
boundary = m2.boundary()
mesh = m2.toMesh()


orientated


In [11]:
# show the boundaries 
Pic = pv.Plotter()
Pic.add_mesh(mesh)
for ele in boundary:
    boundline = np.array([sgm.endP.pos for sgm in ele]+[ele[0].endP.pos])
    Pic.add_lines(boundline, color='green', width=3)
Pic.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [12]:
# calculate harmonic functions in tangent and normal directions  
f, g = conformalProcess.conformalModel_circleStripe(m2)

In [13]:
# show harmonic functions in tangent and normal directions 
Pic = pv.Plotter(shape=(1,2))

Pic.subplot(0,0)
meshtem = mesh.copy()
meshtem.point_data['tangent'] = 2*np.pi*f
Pic.add_mesh(meshtem, scalars='tangent',style='surface',cmap=ListedColormap([black,white]*30))

Pic.subplot(0,1) 
meshtem = mesh.copy()
meshtem.point_data['normal'] = g/g.max()
Pic.add_mesh(meshtem, scalars='normal',style='surface',cmap=ListedColormap([black,white]*60))

Pic.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
# conformal mapping surface onto complex plan
complexValue = g/g.max()+2*np.pi*1j*f
# conformal translate into a loop stripe on complex plan by Exp function 
expValue = np.exp(complexValue)